

## 1. 注意力机制的核心公式是什么？解释每个符号的含义。

$$\text{Attention}(Q,K,V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}} + M\right)V$$

其中含义：

- 计算相似度:`QKᵀ` 计算query和key的匹配度

- **调尺度**:`/√dₖ`
    防止$d_k$较大时点积结果过大导致softmax梯度消失，训练不稳定

- **加掩码**:`+M`  
  - **Padding Mask**：$M_{ij}=-\infty$（忽略无效位置）
  - **Causal Mask**：$M_{ij}=-\infty \ (i<j)$（防止未来信息泄露）

-  **归一化** `softmax(·)`：  
      - 输出非负且和为1的概率分布
      - 指数运算放大显著相关性

-  **加权求和（最终输出）**:`softmax(·) · V`：

## 2. softmax可以改成线性归一化方法吗？
即把$a_i = \frac{e^{s_i}}{\sum_{j=1}^n e^{s_j}}$ 换成  $a_i = \frac{s_i}{\sum_{j=1}^n s_j}$ 

不可以，理由有2:
1. 注意力的核心是“抓重点”，但线性归一化无法强化高分优势：
- 例：分数 $[10,1,1,1]$（总和13）  
  - 线性归一化权重：$[0.77, 0.08, 0.08, 0.08]$（差距小，次要信息仍占比）  
  - softmax归一化权重：$[≈1.0, ≈0, ≈0, ≈0]$（强聚焦关键信息）  

2. 对负分数敏感，易出现异常
- 注意力分数 $QK^T/\sqrt{d_k}$ 可正可负，线性归一化会出问题：  
  - 若分数含负数，总和可能为0（除零错误）或负数（权重为负，无物理意义，“关注度”不能为负）；  
  - softmax通过 $e^{s_i}$ 把任意分数转正数，避免异常。

# 3.标准注意力机制的时间复杂度是多少？

时间复杂度为**O(nmd)**  
当n=m时，复杂度为O(n²d)，意味着序列长度增加10倍，计算量增加100倍。

其中：  
- n: Query序列长度  
- m: Key/Value序列长度  
- d: 特征维度（通常d << n,m）


| 计算步骤           | 矩阵维度            | 计算量              | 时间复杂度   |
|--------------------|---------------------|---------------------|--------------|
| QKᵀ (相似度计算)    | (n×d) × (d×m)       | n×m×d 次乘加        | O(nmd)       |
| Softmax归一化       | (n×m)               | n×m 次exp+除法      | O(nm)        |
| 加权求和 (AV)       | (n×m) × (m×d)       | n×m×d 次乘加        | O(nmd)       |


## 4. 注意力机制空间复杂度多少？

空间复杂度为 **O(n² + nd)**  
其中：  
- `n²` 来自QKᵀ和注意力权重矩阵  
- `nd` 来自Q/K/V和输出矩阵



| 组件          | 空间占用公式       | 示例(n=512,d=768) | 显存占用 |
|---------------|--------------------|-------------------|---------|
| Q/K/V矩阵     | `3 × n × d`        | 3×512×768         | 4.5MB     |
| QKᵀ矩阵       | `n × n`            | 512×512           | 1MB     |
| 注意力权重`softmax(QKᵀ/√d)`     | `n × n`            | 512×512           | 1MB     |
| 输出矩阵 `Attention(Q,K,V)`     | `n × d`            | 512×768           | 1.5MB   |

计算例子：以fp32精度为例，显存占用 = (3 × 512 × 768) × 4 ÷ (1024×1024) = 4.5MB
